In [20]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
from pyLDAvis.gensim import prepare

path_base = "/mnt/nfs/scratch1/hshukla/sentence_results/"
model_base = "sen_lda_{}_{}.model"
data_base = "df_sen_{}_{}.pkl"
#data_base = "df_sen_{}_{}_tmp.csv"
risk_label = "item1a_risk"
mda_label = "item7_mda"
start_year = 2012
end_year = 2015
warnings.filterwarnings("ignore", category=DeprecationWarning)

""" Risk and MDA for window=1,5,7 for a total of 6 viz """

' Risk and MDA for window=1,5,7 for a total of 6 viz '

In [10]:
def create_viz(lda_path, data, label, is_filter_extreme):
    lda_model = LdaModel.load(lda_path)
    data_slice = data[(data.year_x >= start_year) & (data.year_x <= end_year)]
    corpus, dictionary = get_corpus_id2word(data_slice[label].to_list(), is_filter_extreme)
    viz = prepare(lda_model, corpus, dictionary)
    del corpus, dictionary
    return viz

def get_corpus_id2word(document_list, is_filter_extreme):
    documents = [sentence_grp for doc in document_list for sentence_grp in doc]
    dictionary = Dictionary(documents)
    if is_filter_extreme:
        dictionary.filter_extremes(no_below=10)
    corpus = [dictionary.doc2bow(doc) for doc in documents]
    _temp = dictionary[0] # Initialize id2token mappings
    id2word = dictionary.id2token
    return (corpus,dictionary)

# WindowSize = 1, WindowOverlap = 1

In [ ]:
one_data = pd.read_pickle(path_base + data_base.format(1,1))

#### Risk

In [ ]:
# Wait on this to finish processing
# one_data_path = path_base + data_base.format(1,1)
# one_risk_lda_path = path_base + model_base.format(risk_label,1)
# one_mda_lda_path = path_base + model_base.format(mda_label,1)
one_risk_lda_path = path_base + model_base.format(risk_label,1)
one_risk_viz = create_viz(one_risk_lda_path, one_data, risk_label, False)
display(one_risk_viz)

#### MD

In [ ]:
# Wait on this to finish processing
#one_mda_lda_path = path_base + model_base.format(mda_label,1)
#one_mda_viz = create_viz(one_risk_lda_path, one_data, mda_label, False)
#display(one_mda_viz)

# WindowSize = 5, WindowOverlap = 2

In [7]:
five_data = pd.read_pickle(path_base + data_base.format(5,2))

#### Risk

In [11]:
five_risk_lda_path = path_base + model_base.format(risk_label,5)
five_risk_viz = create_viz(five_risk_lda_path, five_data, risk_label, True)
display(five_risk_viz)

AttributeError: 'DataFrame' object has no attribute 'ix'

#### MD&A

In [ ]:
five_mda_lda_path = path_base + model_base.format(mda_label,5)
five_mda_viz = create_viz(five_mda_lda_path, five_data, mda_label, True)
display(five_mda_viz)
del five_data

## WindowSize = 7, WindowOverlap = 3

In [ ]:
seven_data = pd.read_pickle(path_base + data_base.format(7,3))

#### Risk

In [ ]:
seven_risk_lda_path = path_base + model_base.format(risk_label,7)
seven_risk_viz = create_viz(seven_risk_lda_path, seven_data, risk_label, True)
display(seven_risk_viz)

#### MD&A

In [ ]:
seven_mda_lda_path = path_base + model_base.format(mda_label,7)
seven_mda_viz = create_viz(seven_mda_lda_path, seven_data, mda_label, True)
display(seven_mda_viz)
del seven_data